In [1]:
import treeswift as ts
from itertools import product
import pandas as pd

In [2]:
def label_tree(tree):
    for ix, nd in enumerate(tree.traverse_postorder(leaves=True, internal=True)):
        if not nd.is_leaf():
            nd.set_label(f"N{ix}")
    return tree

def distance_between(u, v):
    if not isinstance(u, Node):
        raise TypeError("u must be a Node")
    if not isinstance(v, Node):
        raise TypeError("v must be a Node")
    if u == v:
        return 0.
    elif u == v.parent:
        return v.edge_length
    elif v == u.parent:
        return u.edge_length
    u_dists = {u:0.}; v_dists = {v:0.}
    c = u; p = u.parent # u traversal
    while p is not None:
        u_dists[p] = u_dists[c]
        if c.edge_length is not None:
            u_dists[p] += c.edge_length
        if p == v:
            return u_dists[p]
        c = p; p = p.parent
    c = v; p = v.parent # v traversal
    while p is not None:
        v_dists[p] = v_dists[c]
        if c.edge_length is not None:
            v_dists[p] += c.edge_length
        if p in u_dists:
            return u_dists[p] + v_dists[p]
        c = p; p = p.parent
    raise RuntimeError("u and v are not in the same Tree")

def make_placement_tree(tree, qlist):
    lbl2nd = reft.label_to_node(selection="all")
    qparentlist = [lbl2nd[q].parent.label for q in qlist]
    for nd in tree.traverse_postorder(leaves=True, internal=True):
        parent = nd.parent
        lbl = f"P-{nd.label}"
        if parent is None:
            pnd = ts.Node(label=lbl, edge_length=0)
            nd.set_parent(pnd)
            pnd.add_child(nd)
            reft.root = pnd
        elif not(parent.label in qparentlist):
            parent.remove_child(nd)
            pnd = ts.Node(label=lbl, edge_length=nd.get_edge_length()/2)
            parent.add_child(pnd)
            pnd.set_parent(parent)
            nd.set_edge_length(nd.get_edge_length()/2)
            nd.set_parent(pnd)
            pnd.add_child(nd)
        else:
            pl = (nd.get_edge_length() + parent.get_edge_length())/2
            if pl > nd.get_edge_length():
                nd = nd.parent
                parent = nd.parent
                parent.remove_child(nd)
                pnd = ts.Node(label=lbl, edge_length=pl)
                parent.add_child(pnd)
                pnd.set_parent(parent)
                nd.set_edge_length(nd.get_edge_length()-pl)
                nd.set_parent(pnd)
                pnd.add_child(nd)
            else:
                parent.remove_child(nd)
                pnd = ts.Node(label=f"P-{nd.label}", edge_length=nd.get_edge_length()-pl)
                parent.add_child(pnd)
                pnd.set_parent(parent)
                nd.set_edge_length(pl)
                nd.set_parent(pnd)
                pnd.add_child(nd)
    return tree

In [3]:
reftree = "backbone_blen_all-WoLv2"
ref = "WoLv2"

In [4]:
with open(f"../results/placement_comparison/queries_all-{ref}.txt") as f:
    qlist = list(map(lambda x: x.strip(), f.readlines()))
with open(f"../results/placement_comparison/placements_all-{ref}.txt") as f:
    plist = list(map(lambda x: x.strip(), f.readlines()))

In [22]:
bbt.write_tree_newick(f"../misc_data/backbone_blen_conserved-WoLv2.nwk")

In [23]:
reft = ts.read_tree_newick(f"../misc_data/{reftree}.nwk")
bbt = ts.read_tree_newick(f"../misc_data/backbone_blen_conserved-WoLv2.nwk")
blenmap = {}
for a in bbt.traverse_postorder():
    n = reft.mrca(bbt.extract_subtree(a).labels(internal=False))
    print(n.get_edge_length())
    if n.get_edge_length() is not None:
        a.set_edge_length(n.get_edge_length())
        blenmap[a.label] = a.get_edge_length()

0.0641039312
0.058705343400000005
0.0460288103
0.0482649762
0.0
0.0
0.0380878342
0.0
0.0
0.0
1e-06
0.0757476102
0.0268526221
0.0417796244
0.509325
0.08459293840000001
0.0778422724
0.0380164717
0.0294148112
0.0019815207
0.0037662202
0.0306683067
0.0407333719
0.0390519794
0.0514704793
0.0
0.0
0.0
0.0084705741
0.0620746063
0.0415142299
0.0109589716
0.023226611
0.0176286348
0.0052403127
0.0039287306
0.0233444988
0.0334427979
0.0487808795
0.0424997322
0.0806123109
0.0
0.0
0.083115444
0.0665663542
0.0498253022
0.047171847
0.0286574742
0.0447484931
0.0212567542
0.0171218039
0.0266586813
0.0243563676
0.0392262366
0.0528834321
0.09729878830000001
0.0855508336
0.081727174
0.0165333145
0.0243539048
0.030702334
0.0299500476
0.0261159952
0.0255792653
0.0236317029
0.0186534594
0.0247416654
0.0229447455
0.0260725981
0.021441279
0.0275396422
0.0290841384
0.0240437879
0.0431024133
0.0280685855
0.0413933697
0.0390828784
0.0137334377
0.0239324206
0.0343004049
0.0139924798
0.0181164585
0.0
0.0
0.053469729

KeyboardInterrupt: 

In [21]:
for a in reft.traverse_postorder():
    if a.get_edge_length() != blenmap.get(a.label, False):
        print(a.label)
        

0.48
0.39
0.42
0.41
0.29
0.41
0.42
0.31
0.45
0.35
0.33
0.28
0.37
0.46
0.41
0.15
0.38
0.29
0.37
0.43
0.42
0.49
0.49
0.48
0.38
0.47
0.35
0.37
0.43
0.29
0.31
0.4
0.39
0.4
0.46
0.42
0.39
0.38
0.41
0.45
0.49
0.39
0.46
0.8
0.4
0.03
0.42
0.21
0.15
0.33
0.43
0.17
0.03
0.26
0.46
0.4
0.45
0.47
0.43
0.5
0.49
0.44
0.39
0.45
0.45
0.32
0.53
0.28
0.44
0.47
0.33
0.14
0.45
0.5
0.3
0.34
0.46
0.39
0.43
0.37
0.33
0.47
0.23
0.33
0.46
0.42
0.51
0.42
0.12
0.39
0.45
0.44
0.26
0.46
0.46
0.39
0.42
0.46
0.5
0.34
0.49
0.29
0.38
0.44
0.34
0.5
0.43
0.49
0.41
0.39
0.35
0.37
0.32
0.33
0.5
0.44
0.54
0.14
0.39
0.4
0.1
0.48
0.45
0.31
0.42
0.49
0.14
0.42
0.49
0.59
0.47
0.33
0.35
0.26
0.43
0.36
0.16
0.45
0.42
0.3
0.67
0.58
0.35
0.07
0
0.42
0.16
0.44
0.41
0.37
0.13
0.48
0.27
0.45
0
0.01
0.13
0.3
0.28
0.09
0.26
0
0.36
0.39
0.32
0.5
0.4
0.47
0.45
0.36
0.8
0.4
0.29
0.65
0.49
0.34
0.23
0.01
0.41
0.96
0.66
0.39
0.44
0.33
0.33
0.46
0.3
0.26
0.07
0.32
0.46
0.4
0.36
0.46
0.12
0.4
0.47
0.39
0.38
0.4
0.43
0.33
0.4
0.26
0.43
0.24
0.2

In [6]:
reft = ts.read_tree_newick(f"../misc_data/{reftree}.nwk")
# reft.suppress_unifurcations()
# reft.resolve_polytomies()
# reft.write_tree_newick(f"../../misc_data/backboneU-{reftree}.nwk")
lbl2nd = reft.label_to_node(selection="all")

In [7]:
ntree = ts.read_tree_newick(f"../misc_data/{reftree}.nwk")
# ntree.suppress_unifurcations()
# ntree.resolve_polytomies()
# ntree.write_tree_newick(f"../../misc_data/backbone-{reftree}.nwk")

In [8]:
for a, b in zip(ntree.traverse_postorder(internal=False), reft.traverse_postorder(internal=False)):
    if a.label is None:
        a.label = "N"
    if b.label is None:
        b.label = "N"
    if (a.label[0] == "G" or b.label[0] == "G") and (a.label[0] != b.label[0]):
        print(a.label, b.label)
        break

In [9]:
prunedict = {}
for q in qlist:
    nd_new = lbl2nd[q].parent.parent
    # nd_new = [nd for nd in lbl2nd[q].parent.child_nodes() if nd.label != q][0]
    prunedict[lbl2nd[q].parent.label] = nd_new.label

In [102]:
d = list(map(
    lambda x: reft.distance_between(
        lbl2nd[prunedict.get(x[0], x[0])],
        lbl2nd[x[1]].parent),
    list(product(plist, qlist)))
)
dfd = pd.DataFrame(
    {
        "p": [pq[0] for pq in list(product(plist, qlist))],
        "q": [pq[1] for pq in list(product(plist, qlist))],
        "d": d
    }
)
dfd.to_csv(f"../misc_data/distance_map-{reftree}.tsv", sep="\t", index=False)

In [10]:
reft = ts.read_tree_newick(f"../misc_data/{reftree}.nwk")
reft = make_placement_tree(reft, qlist)
lbl2nd = reft.label_to_node(selection="all")
d = list(map(
    lambda x: reft.distance_between(
        lbl2nd[f"P-{prunedict.get(x[0], x[0])}"],
        lbl2nd[x[1]].parent),
    list(product(plist, qlist)))
)
dfd = pd.DataFrame(
    {
        "p": [pq[0] for pq in list(product(plist, qlist))],
        "q": [pq[1] for pq in list(product(plist, qlist))],
        "d": d
    }
)
dfd.to_csv(f"../misc_data/mdistance_map-{reftree}.tsv", sep="\t", index=False)

In [ ]:
reft = ts.read_tree_newick(f"../../misc_data/backbone-{reftree}.nwk")
lbl2nd = reft.label_to_node(selection="all")

d = list(map(
    lambda x: reft.distance_between(
        lbl2nd[x[0]],
        lbl2nd[x[1]]),
    list(product(plist, qlist)))
)
dfd = pd.DataFrame(
    {
        "p": [pq[0] for pq in list(product(plist, qlist))],
        "q": [pq[1] for pq in list(product(plist, qlist))],
        "d": d
    }
)

In [ ]:
dfd = dfd[dfd["p"].str.startswith("G")]
dfd = dfd[dfd["p"] != dfd["q"]]
dfn = dfd.loc[dfd.groupby('q').d.idxmin()].reset_index(drop=True)
# dfn.to_csv(f"../../misc_data/distance_closest{reftree}.tsv", sep="\t", index=False)